In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import caer
import canaro
import cv2 as cv
import gc
import os

In [ ]:
IMG_SIZE = (80, 80)
channel = 1
char_path = 'Simpsons'

In [ ]:
# Grab top 10 elements of data
char_dct = {}
for char in os.listdir(char_path):
    char_dct[char] = len(os.listdir(os.path.join(char_path, char)))

# Sort
char_dct = caer.sort_dict(char_dct, descending=True)

In [ ]:
characters = []
count = 0
for i in char_dct:
    count += 1
    characters.append(i[0])
    if(count>10): break

In [ ]:
# Create training data
train = caer.preprocess_from_dir(char_path, characters, channels=channel, IMG_SIZE = IMG_SIZE, isShuffle= True)

In [ ]:
plt.figure(figsize(30,30))
plt.imshow(train[0][0], cmap = 'gray)
plt.show()

In [ ]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical
# Normalization
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [ ]:
x_train, x_test, y_train, y_test = caer.train_val_split(featureSet, labels, val_ratio=0.2)

In [ ]:
# Free up the memory
del train
del featureSet
del labels
gc.collect()

In [ ]:
# Image data generetor
datagen = camero.generetors.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size = 30)

In [ ]:
model = canero.models.createSimpsonsModel(IMG_SIZE = IMG_SIZE, channels = channel, out_dim = len(characters), loss='binary_crossentropy', decay = 1e-6, learning_rate = .001, momentum = 0.9, nesterov = True)

In [ ]:
model.summary()

In [ ]:

from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]
training = model.fit(train_gen,
                    steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks = callbacks_list)

In [ ]:
test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/charles_montgomery_burns_0.jpg'

img = cv.imread(test_path)

plt.imshow(img)
plt.show()

In [ ]:

def prepare(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, IMG_SIZE)
    image = caer.reshape(image, IMG_SIZE, 1)
    return image

In [ ]:
predictions = model.predict(prepare(img))


In [ ]:
# Getting class with the highest probability
print(characters[np.argmax(predictions[0])])